In [33]:
import json #importing Json Reader
import numpy as np
import pandas as pd

In [34]:
json_data=open('data.json').read()
data = json.loads(json_data) #Reading data from Json file

In [35]:
data

[{'tr': 'bu adam', 'en': 'this man'},
 {'tr': 'bu kedi', 'en': 'this cat'},
 {'tr': 'bu iyi adam', 'en': 'this good man'},
 {'tr': 'bu iyi kedi', 'en': 'this good cat'},
 {'tr': 'iyi araba', 'en': 'good car'}]

In [36]:
#pushing english and turkish sentences into arrays respectively
english_sentences = []
turkish_sentences = []
for sent in data:
        english_sentences.append(sent['en'])
        turkish_sentences.append(sent['tr'])


In [37]:
turkish_sentences 

['bu adam', 'bu kedi', 'bu iyi adam', 'bu iyi kedi', 'iyi araba']

In [38]:
english_sentences

['this man', 'this cat', 'this good man', 'this good cat', 'good car']

In [41]:
#for i in range(len(english_sentences)):
    #nuevo11 = english_sentences[i]
   # print(nuevo11)
   # nueve11 = nuevo11.split()
   # print(nueve11)
   # yeni11 = turkish_sentences[i]
   # print(yeni11)
   # yeni111 = yeni11.split()
    #print(yeni111)

In [39]:
#getting unique(unrepeated) words from the data and putting them into turkish and english arrays respectively
unrepeated_turkish_words = []
unrepeated_english_words = []
for i in range(len(data)):
    eng = data[i]['en'].split(' ') #get english sentence
    trk = data[i]['tr'].split(' ')#get turkish sentence
    
    for j in range(len(trk)):#for all turkish sentences
        if trk[j] not in unrepeated_turkish_words:
            unrepeated_turkish_words.append(trk[j])#get word
    for j in range(len(eng)):#for all english sentences
        if eng[j] not in unrepeated_english_words:
            unrepeated_english_words.append(eng[j])#get word

In [40]:
unrepeated_turkish_words

['bu', 'adam', 'kedi', 'iyi', 'araba']

In [41]:
unrepeated_english_words

['this', 'man', 'cat', 'good', 'car']

In [42]:
unrepeated_turkish_words = list(unrepeated_turkish_words)#putting turkish words into a list

In [43]:
unrepeated_turkish_words

['bu', 'adam', 'kedi', 'iyi', 'araba']

In [44]:
unrepeated_english_words = list(unrepeated_english_words)#putting english words into a list
number_turkish_words = len(unrepeated_turkish_words)#number of turkish words

In [45]:
from collections import defaultdict

In [46]:
translation_probability = dict()#Initializing dictionary
for en_word in unrepeated_english_words:
    translation_probability[en_word] = dict()#open dictionary for all english words
    
    for tr_word in unrepeated_turkish_words:
        translation_probability[en_word][tr_word] = 1 / number_turkish_words#T(eng|trk) is assigned such that all alignments are equally likely

In [47]:
translation_probability

{'this': {'bu': 0.2, 'adam': 0.2, 'kedi': 0.2, 'iyi': 0.2, 'araba': 0.2},
 'man': {'bu': 0.2, 'adam': 0.2, 'kedi': 0.2, 'iyi': 0.2, 'araba': 0.2},
 'cat': {'bu': 0.2, 'adam': 0.2, 'kedi': 0.2, 'iyi': 0.2, 'araba': 0.2},
 'good': {'bu': 0.2, 'adam': 0.2, 'kedi': 0.2, 'iyi': 0.2, 'araba': 0.2},
 'car': {'bu': 0.2, 'adam': 0.2, 'kedi': 0.2, 'iyi': 0.2, 'araba': 0.2}}

In [48]:
total_iterations = 5
for i in range(total_iterations):
    count_e_given_f = dict() #intialaizing counts and totals dictis
    total_f = dict()
    print("Iteration Number = ", i)
    #initialize
    for en_word in unrepeated_english_words:
        count_e_given_f[en_word] = dict()#open a counts dictionary for all english words
        for tr_word in unrepeated_turkish_words:
            count_e_given_f[en_word][tr_word] = 0 #set C(eng|turk) = 0 for each iteration
            #  Setting up total(f) = 0 for all f
    for tr_word in unrepeated_turkish_words:
        total_f[tr_word] = 0
        
                #do for all sentence pairs
    
    for j in range(len(english_sentences)):#for all english sentences
        eng_sent = english_sentences[j]#get sentence by index
        eng_sent = eng_sent.split()#split sentence to get words
        trk_sent = turkish_sentences[j]#get corresponding turkish sentence
        trk_sent = trk_sent.split()#split to get words

         # Computing Normalisation
       
        s_total = dict() #initializing dict
        for e in eng_sent: #for every words in english sentence
            #print(e)
            s_total[e] = 0 #set value to 0 for each iteration
            for f in trk_sent:# fro every word in turkish sentence
                #assign the sum of the values of the translation probability e given the various values for f,this is essentially
                #computing P(e|a,f)
                #print(f)
                s_total[e] += translation_probability[e][f]
                #print(s_total)
        
        
        #collecting counts        
        for e in eng_sent: #for word in english sentence
            #print(e)
            for f in trk_sent:#for words in turkish sentnce
                #print(f)
                #Count(eng|trk) is equal to T(eng|trk) divided by P(eng|a,turk) and summed over for all values(words) of trk
                count_e_given_f[e][f] += translation_probability[e][f] / s_total[e]  #This division Computes P(a|e,f)
                total_f[f] += translation_probability[e][f] / s_total[e] #same is done to get counts for all turkish words
                #print(count_e_given_f)
                #print(total_f)
                

                 #this section estimates probabilities           
    for f in unrepeated_turkish_words: #for each unique turkish word 
        for e in unrepeated_english_words:#for each unique english word 
            # update T(eng|trk) with the result of Count(eng|trk) divided by Count(trk)
            # for example T(this|bu) = Count(this|bu){evidence that this translates to bu} / count(bu){total number of evidence for word bu}
            translation_probability[e][f] = count_e_given_f[e][f] / total_f[f]
            #print(translation_probability[e][f], i)
            #print("Iteration Number = ", i)
    print("s_total(e)")
    print(s_total)
    print("Counts(e|f)")
    print(count_e_given_f)
    print("total(f)")
    print(total_f)
    print("t(e|f)")
    print(translation_probability)

      

Iteration Number =  0
s_total(e)
{'good': 0.4, 'car': 0.4}
Counts(e|f)
{'this': {'bu': 1.6666666666666665, 'adam': 0.8333333333333333, 'kedi': 0.8333333333333333, 'iyi': 0.6666666666666666, 'araba': 0}, 'man': {'bu': 0.8333333333333333, 'adam': 0.8333333333333333, 'kedi': 0, 'iyi': 0.3333333333333333, 'araba': 0}, 'cat': {'bu': 0.8333333333333333, 'adam': 0, 'kedi': 0.8333333333333333, 'iyi': 0.3333333333333333, 'araba': 0}, 'good': {'bu': 0.6666666666666666, 'adam': 0.3333333333333333, 'kedi': 0.3333333333333333, 'iyi': 1.1666666666666665, 'araba': 0.5}, 'car': {'bu': 0, 'adam': 0, 'kedi': 0, 'iyi': 0.5, 'araba': 0.5}}
total(f)
{'bu': 4.000000000000001, 'adam': 1.9999999999999998, 'kedi': 1.9999999999999998, 'iyi': 3.0, 'araba': 1.0}
t(e|f)
{'this': {'bu': 0.4166666666666665, 'adam': 0.4166666666666667, 'kedi': 0.4166666666666667, 'iyi': 0.2222222222222222, 'araba': 0.0}, 'man': {'bu': 0.20833333333333326, 'adam': 0.4166666666666667, 'kedi': 0.0, 'iyi': 0.1111111111111111, 'araba': 0.

In [17]:
translation_probability

{'this': {'bu': 0.8053005206083633,
  'adam': 0.23933560832990564,
  'kedi': 0.23933560832990564,
  'iyi': 0.05238565460522278,
  'araba': 0.0},
 'man': {'bu': 0.07882444742590902,
  'adam': 0.7496529444365608,
  'kedi': 0.0,
  'iyi': 0.006014944141431334,
  'araba': 0.0},
 'cat': {'bu': 0.07882444742590902,
  'adam': 0.0,
  'kedi': 0.7496529444365608,
  'iyi': 0.006014944141431334,
  'araba': 0.0},
 'good': {'bu': 0.037050584539818514,
  'adam': 0.011011447233533509,
  'kedi': 0.011011447233533509,
  'iyi': 0.9229546021564303,
  'araba': 0.18636970110664777},
 'car': {'bu': 0.0,
  'adam': 0.0,
  'kedi': 0.0,
  'iyi': 0.012629854955484332,
  'araba': 0.8136302988933521}}

In [18]:
#import NLTK and IBMModel1 Module
import nltk
from nltk.translate import AlignedSent, IBMModel1

In [19]:
bitext= []
#loading in equivalent aligned sentences aligned 
bitext.append(AlignedSent(["bu","adam"],
["this","man"]))
bitext.append(AlignedSent(["bu", "kedi"],
["this", "cat"]))
bitext.append(AlignedSent(["bu", "iyi", "adam"],
["this", "good", "man"]))
bitext.append(AlignedSent(["bu", "iyi", "kedi"], ["this", "good", "cat"]))
bitext.append(AlignedSent(["iyi", "araba"], ["good", "car"]))
print(bitext)
myIBM = IBMModel1(bitext,5)# 5 iterations

[AlignedSent(['bu', 'adam'], ['this', 'man'], Alignment([])), AlignedSent(['bu', 'kedi'], ['this', 'cat'], Alignment([])), AlignedSent(['bu', 'iyi', 'adam'], ['this', 'good', 'man'], Alignment([])), AlignedSent(['bu', 'iyi', 'kedi'], ['this', 'good', 'cat'], Alignment([])), AlignedSent(['iyi', 'araba'], ['good', 'car'], Alignment([]))]


In [20]:
#print translation probabilities
print(myIBM.translation_table["bu"]["this"])
print(myIBM.translation_table["adam"]["man"])
print(myIBM.translation_table["kedi"]["cat"])
print(myIBM.translation_table["iyi"]["good"])
print(myIBM.translation_table["araba"]["car"])


0.7378729536520211
0.8160370095326962
0.8160370095326962
0.9209954720615414
0.8610884003804863
